In [ ]:
import requests
import csv
from datetime import datetime

# Your Aviation Edge API key
API_KEY = 'MY_API_KEY'

# Define the API endpoint
url = "https://aviation-edge.com/v2/public/flightsHistory"

# Define the parameters
params = {
    'key': API_KEY,
    'code': 'SFO',                  # IATA code for San Francisco International Airport
    'type': 'departure',            # Type of flights: departures
    'date_from': '2024-05-31',      # Start date for the flights
    'date_to': '2024-06-01',        # End date for the flights
    'airline_iata': 'UA'            # IATA code for United Airlines
}

# Make the GET request
response = requests.get(url, params=params)

# Print the entire response for debugging
print("Status Code:", response.status_code)
print("Response Text:", response.text)

# Check if the request was successful
if response.status_code == 200:
    # Parse the JSON response
    flights = response.json()
    print("Parsed JSON:", flights)  # Print parsed JSON for debugging

    # Check if there are any flights
    if flights and isinstance(flights, list):  # Ensure flights is a list
        # Prepare data for CSV
        flight_data = []
        for flight in flights:
            # Check if flight is a dictionary
            if isinstance(flight, dict):
                flight_info = {
                    'Flight Number': flight.get('flight', {}).get('iataNumber', 'N/A') if isinstance(flight.get('flight'), dict) else 'N/A',
                    'Scheduled Departure Time': flight.get('departure', {}).get('scheduledTime', 'N/A') if isinstance(flight.get('departure'), dict) else 'N/A',
                    'Actual Departure Time': flight.get('departure', {}).get('actualTime', 'N/A') if isinstance(flight.get('departure'), dict) else 'N/A',
                    'Status': flight.get('status', 'N/A')
                }
                flight_data.append(flight_info)

        # Define the CSV file name
        csv_file_name = 'SFO_Departures_United_Airlines_2024-05-31.csv'

        # Write data to CSV
        with open(csv_file_name, mode='w', newline='') as csv_file:
            fieldnames = ['Flight Number', 'Scheduled Departure Time', 'Actual Departure Time', 'Status']
            writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

            writer.writeheader()
            for row in flight_data:
                writer.writerow(row)

        print(f"Data successfully written to {csv_file_name}")
    else:
        print("No flights found for the specified criteria or response is not a list.")
else:
    print(f"Error: {response.status_code}, {response.text}")


Status Code: 200
Response Text: [{"type": "departure", "status": "active", "departure": {"iataCode": "sfo", "icaoCode": "ksfo", "terminal": "3", "gate": "f11", "delay": 122, "scheduledTime": "2024-05-30t17:00:00.000", "estimatedTime": "2024-05-30t19:02:00.000"}, "arrival": {"iataCode": "hnl", "icaoCode": "phnl", "terminal": "2", "scheduledTime": "2024-05-30t19:35:00.000"}, "airline": {"name": "united airlines", "iataCode": "ua", "icaoCode": "ual"}, "flight": {"number": "1509", "iataNumber": "ua1509", "icaoNumber": "ual1509"}}, {"type": "departure", "status": "active", "departure": {"iataCode": "sfo", "icaoCode": "ksfo", "terminal": "3", "gate": "e9", "delay": 23, "scheduledTime": "2024-05-30t17:20:00.000", "estimatedTime": "2024-05-30t17:38:00.000", "actualTime": "2024-05-30t17:42:00.000", "estimatedRunway": "2024-05-30t17:42:00.000", "actualRunway": "2024-05-30t17:42:00.000"}, "arrival": {"iataCode": "koa", "icaoCode": "phko", "terminal": "1", "baggage": "a1", "gate": "2", "scheduledT

In [ ]:
import csv
from datetime import datetime

# Function to calculate delay status
def calculate_delay_status(delta):
    if delta >= 15:
        return 'Delayed'
    else:
        return 'On Time'

# Read data from the previously generated CSV file
csv_file_name = 'SFO_Departures_United_Airlines_2024-05-31.csv'

flight_data = []
with open(csv_file_name, mode='r', newline='') as csv_file:
    reader = csv.DictReader(csv_file)
    for row in reader:
        # Parse datetime strings to datetime objects for calculation
        scheduled_time_str = row['Scheduled Departure Time']
        actual_time_str = row['Actual Departure Time']

        if scheduled_time_str != 'N/A' and actual_time_str != 'N/A':
            scheduled_time = datetime.fromisoformat(scheduled_time_str)
            actual_time = datetime.fromisoformat(actual_time_str)

            # Calculate delta in minutes
            delta = (actual_time - scheduled_time).total_seconds() // 60

            # Calculate delay status
            delay_status = calculate_delay_status(delta)
        else:
            # If scheduled or actual time is missing or 'N/A'
            delta = 'N/A'
            delay_status = 'N/A'

        # Add delay information to the row
        row['Delay (minutes)'] = delta
        row['Delay Status'] = delay_status

        # Append modified row to flight_data
        flight_data.append(row)

# Define the new CSV file name for updated data
updated_csv_file_name = 'SFO_Departures_United_Airlines_2024-05-31_updated.csv'

# Write updated data to a new CSV file
with open(updated_csv_file_name, mode='w', newline='') as updated_csv_file:
    fieldnames = ['Flight Number', 'Scheduled Departure Time', 'Actual Departure Time', 'Status', 'Delay (minutes)', 'Delay Status']
    writer = csv.DictWriter(updated_csv_file, fieldnames=fieldnames)

    writer.writeheader()
    for row in flight_data:
        writer.writerow(row)

print(f"Updated data successfully written to {updated_csv_file_name}")


Updated data successfully written to SFO_Departures_United_Airlines_2024-05-31_updated.csv
